<a href="https://colab.research.google.com/github/shiki-fujin/DS-MLE-exam/blob/main/text_classification_sst2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initializations

In [1]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [2]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [39]:
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.3 MB/s eta 0:00:00


In [3]:
! pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 40.7 MB/s eta 0:00:00


In [4]:
import random
import pandas as pd
import numpy as np

In [40]:
from huggingface_hub import notebook_login
import transformers
import datasets
import evaluate
from ray import tune

In [6]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


## Fine-tuning another dataset provided : **SST-2** (*sentiment analysis*)


In [61]:
# seed for reproducibility
random_seed = 30

### Prepare dataset

In [63]:
def encode_sst2(examples):
    return tokenizer(examples['sentence'], truncation=True)


def load_data(task):
  dataset = datasets.load_dataset('glue', task)
  metric = evaluate.load('glue', task)
  
  encoded_dataset = dataset.map(encode_sst2, batched=True)
  train_dataset = encoded_dataset['train'].shuffle(seed=random_seed).select(range(850))
  eval_dataset = encoded_dataset['validation'].shuffle(seed=random_seed).select(range(850))
  return train_dataset, eval_dataset, metric


actual_task = 'sst2'
pretrained_model_name = 'distilbert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(pretrained_model_name, use_fast=True)
train_dataset, eval_dataset, metric = load_data(actual_task)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Fine tuning using default args

In [47]:
def model_init():
    return transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2)

In [48]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [64]:
metric_name = 'accuracy'
batch_size = 16
training_output_name = f'{pretrained_model_name}-finetuned-{actual_task}'

benchmark_args = transformers.TrainingArguments(
    training_output_name,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True)

benchmark_trainer = transformers.Trainer(
    model_init = model_init,
    args = benchmark_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1

In [65]:
benchmark_trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.441769,0.795294
2,No log,0.421680,0.842353
3,No log,0.445431,0.850588


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 850
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-sst2/checkpoint-54
Configuration saved in distilbert-base-uncased-finetuned-sst2/checkpoint-54/config.json
Model weights saved in distilbert-base-uncased-finetuned-sst2/checkpoint-54/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-sst2/checkpoint-54/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst2/checkpoint-54/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-sst2/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst2/special_

TrainOutput(global_step=162, training_loss=0.28220360367386427, metrics={'train_runtime': 866.3196, 'train_samples_per_second': 2.943, 'train_steps_per_second': 0.187, 'total_flos': 23185416920232.0, 'train_loss': 0.28220360367386427, 'epoch': 3.0})

In [67]:
benchmark_trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 850
  Batch size = 16


{'eval_loss': 0.4216795861721039,
 'eval_accuracy': 0.8423529411764706,
 'eval_runtime': 68.1845,
 'eval_samples_per_second': 12.466,
 'eval_steps_per_second': 0.792,
 'epoch': 3.0}

In [66]:
benchmark_trainer.push_to_hub()

Saving model checkpoint to distilbert-base-uncased-finetuned-sst2
Configuration saved in distilbert-base-uncased-finetuned-sst2/config.json
Model weights saved in distilbert-base-uncased-finetuned-sst2/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-sst2/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst2/special_tokens_map.json


Upload file runs/Jan23_06-39-15_1ef5a0f51472/events.out.tfevents.1674455962.1ef5a0f51472.315.10: 100%|########…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-sst2
   ee4bda8..c715bc0  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-sst2
   ee4bda8..c715bc0  main -> main

To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-sst2
   c715bc0..4ce09cd  main -> main

   c715bc0..4ce09cd  main -> main



'https://huggingface.co/naksu/distilbert-base-uncased-finetuned-sst2/commit/c715bc03ffb87047998c751e51bf5912e62948b1'

Performing Hyper-param search to try to improve these metrics from benchmark :
1.  Validation Loss = 0.422 (reduce)
2.  Accuracy	= 0.842 (increase)


